In [84]:
import unittest
import numpy as np 
import importlib
import sys
sys.path.append('../')
from src import dissipation_tensor, pauli_basis_class #should use a mock instead but since this class has been fully tested its ok
importlib.reload(dissipation_tensor)
importlib.reload(pauli_basis_class)
from src.dissipation_tensor import Dissipation_Tensor
from src.pauli_basis_class import Pauli_Basis
from scipy.sparse import lil_matrix

In [85]:
class Dissipation_tests(unittest.TestCase):

    def test_dissipation_tensor_constructed_with_correct_size_and_initially_zeros(self):

        n = 2
        complex_sc =  [lil_matrix((n, n)) for _ in range(n)]
        my_dissipation_tensor = Dissipation_Tensor("general", complex_sc)
        d_tensor = my_dissipation_tensor.tensor
        first_layer_size = len(d_tensor)
        second_layer_size = len(d_tensor[0])
        inner_matrix_shape = np.shape(d_tensor[0][0].toarray())
        self.assertEqual(first_layer_size, n)
        self.assertEqual(second_layer_size, n)
        self.assertEqual(inner_matrix_shape, (n,n))

    def test_creates_correct_tensor_select_type_of_Kossakowski_matrix_with_getter(self):
        
        n = 2 
        complex_sc = [lil_matrix((n, n)) for _ in range(n)]
        kossakowski_types = ["General", "Symmetric"] #can include diagonal but it's not a priority right now
        my_dissipation_tensor_general = Dissipation_Tensor(kossakowski_types[0], complex_sc)
        my_dissipation_tensor_symmetric = Dissipation_Tensor(kossakowski_types[1], complex_sc)

        self.assertEqual(kossakowski_types[0].lower(), my_dissipation_tensor_general.kossakowski_type)
        self.assertEqual(kossakowski_types[1].lower(), my_dissipation_tensor_symmetric.kossakowski_type)

    def test_validation_of_kossakowski_types(self):

        n = 2
        complex_sc = [lil_matrix((n, n)) for _ in range(n)]
        
        with self.assertRaises(Exception) as context:
            Dissipation_Tensor("Not a kossakowski matrix, just a random string", complex_sc)
        
        self.assertEqual(str(context.exception), "Kossakowski matrix type must be \"general\" or \"symmetric\"")

    def test_complex_strucutre_constants_validation_must_be_list(self):
        complex_sc = "string"

        with self.assertRaises(TypeError) as context:
            Dissipation_Tensor("symmetric", complex_sc)
        
        self.assertEqual(str(context.exception), "complex_structure_constants should be a list of n matrices with dimension n x n")

    def test_complex_strucutre_constants_validation_no_empty_list(self):
        complex_sc = []
        with self.assertRaises(ValueError) as context:
            Dissipation_Tensor("symmetric", complex_sc)
        
        self.assertEqual(str(context.exception), "complex_structure_constants cannot be an empty list")

    
    def test_complex_structure_constants_validation_should_be_list_of_matrices(self):
        complex_sc = ["beep", "boop"]
        with self.assertRaises(TypeError) as context:
            Dissipation_Tensor("symmetric", complex_sc)
        
        self.assertEqual(str(context.exception), "complex_structure_constants must be a list of matrices")

    def test_complex_structure_constants_validation_should_be_list_of_matrices_of_same_dimension_as_list(self):
        complex_sc = [np.zeros((2,2)), np.zeros((2,2)), np.zeros((2,2))]
        with self.assertRaises(ValueError) as context:
            Dissipation_Tensor("symmetric", complex_sc)
        
        self.assertEqual(str(context.exception), f"All matrices in complex_structure_constants must have dimensions ({len(complex_sc)}, {len(complex_sc)}). "
                                 f"Found a matrix with dimensions {complex_sc[0].shape}.")

    def test_build_general_d_tensor_for_su_4(self):
        n = 4
        my_pauli_basis = Pauli_Basis(4)
        
        complex_structure_constants  = my_pauli_basis.complex_structure_constants()  
    
        my_dissipation_tensor = Dissipation_Tensor("general", complex_structure_constants)

        output = my_dissipation_tensor.tensor
        expected = np.array([[0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
        0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
        0. +0.j],[0. +0.j, 0.5+0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
        0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
        0. +0.j],[0. +0.j, 0. +0.j, 0.5+0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
        0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
        0. +0.j],[0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
        0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
        0. +0.j],[0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
        0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
        0. +0.j],[0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0.5+0.j, 0. +0.j,
        0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
        0. +0.j],[0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0.5+0.j,
        0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
        0. +0.j],[0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
        0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
        0. +0.j],[0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
        0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
        0. +0.j],[0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
        0. +0.j, 0. +0.j, 0.5+0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
        0. +0.j],[0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
        0. +0.j, 0. +0.j, 0. +0.j, 0.5+0.j, 0. +0.j, 0. +0.j, 0. +0.j,
        0. +0.j],[0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
        0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
        0. +0.j],[0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
        0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
        0. +0.j],[0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
        0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0.5+0.j,
        0. +0.j],[0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
        0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
        0.5+0.j]])

        self.assertTrue(np.array_equal(output[0][0].toarray(), expected))




        



        


In [86]:
unittest.main(argv=[''], exit = False)

C:\Users\waqas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\scipy\sparse\_lil.py:258: ComplexWarning: Casting complex values to real discards the imaginary part
  x = self.dtype.type(x)
........
----------------------------------------------------------------------
Ran 8 tests in 29.448s

OK


In [87]:
n = 4
my_pauli_basis = Pauli_Basis(4)

complex_structure_constants  = my_pauli_basis.complex_structure_constants()  

C:\Users\waqas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\scipy\sparse\_lil.py:258: ComplexWarning: Casting complex values to real discards the imaginary part
  x = self.dtype.type(x)


In [88]:
my_dissipation_tensor = Dissipation_Tensor("general", complex_structure_constants)